In [13]:
import numpy
import datetime as dt
from astral import LocationInfo
from astral.location import Location
from astral.geocoder import database, group, lookup_in_group
import pytz
from astral.sun import sun
from pydantic import BaseModel

In [41]:
# calculate sunrise for Berlin
# name and region can be any value you like
city=LocationInfo(name="Berlin", region="Germany", timezone="Europe/Berlin", 
                  latitude=52.547302, longitude=13.390178)

In [42]:
berlin = Location(city)

In [43]:
berlin.golden_hour()

(datetime.datetime(2021, 2, 28, 6, 33, 16, 427625, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
 datetime.datetime(2021, 2, 28, 7, 40, 49, 756620, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>))

In [44]:
# Sun dates for tomorrow
sun_params = berlin.sun(date=dt.date(2021, 2, 28))
sun_params

{'dawn': datetime.datetime(2021, 2, 28, 6, 20, 14, 345088, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
 'sunrise': datetime.datetime(2021, 2, 28, 6, 55, 13, 194125, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
 'noon': datetime.datetime(2021, 2, 28, 12, 19, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
 'sunset': datetime.datetime(2021, 2, 28, 17, 43, 30, 383792, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
 'dusk': datetime.datetime(2021, 2, 28, 18, 18, 34, 568493, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>)}

In [11]:
# convert datetime object to google cal iso format
sun_params['dawn'].isoformat()

'2021-02-22T06:32:53.728316+01:00'

In [7]:
# this is a dataclass defined in __init__.py!
type(city)

astral.LocationInfo

## Obtain location info from db of this package

In [2]:
# imports the database of cities and associated timezones, longitude & latitude and so on
db=database()

In [3]:
# dict with keys being European cities and the values are LocationInfo objects!!
europe = group('Europe', db)

In [4]:
lookup_in_group('Berlin', europe)

LocationInfo(name='Berlin', region='Germany', timezone='Europe/Berlin', latitude=52.5, longitude=13.416666666666666)

In [5]:
europe['berlin']

[LocationInfo(name='Berlin', region='Germany', timezone='Europe/Berlin', latitude=52.5, longitude=13.416666666666666)]

In [6]:
db.keys()

dict_keys(['asia', 'africa', 'australia', 'europe', 'indian', 'pacific', 'america', 'etc', 'brazil', 'utc', 'us', 'atlantic'])

## How much information do we need in LocationInfo?

In [24]:
info=LocationInfo(timezone="Europe/Berlin", latitude=52.547302, longitude=13.390178)
berlin = Location(info=info)

In [25]:
# not setting name and region does not hurt, but sets defaults Greenwich England, which might be confusing if shown
# as output somewhere in the process
berlin

Greenwich/England, tz=Europe/Berlin, lat=52.55, lon=13.39

In [39]:
berlin.sunrise(date=dt.date(2021,3,1))

datetime.datetime(2021, 3, 1, 6, 52, 59, 599100, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>)

In [40]:
berlin.golden_hour(date=dt.date(2021,3,1))

(datetime.datetime(2021, 3, 1, 6, 31, 5, 252877, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
 datetime.datetime(2021, 3, 1, 7, 38, 27, 179397, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>))

In [45]:
# try this for a different location and validate results online
info=LocationInfo(timezone='America/Los_Angeles', latitude=37.485215, longitude=-122.236355)
redwoodcity = Location(info)

In [46]:
# yes, seems to be correct!
redwoodcity.sunrise(date=dt.date(2021,3,1))

datetime.datetime(2021, 3, 1, 6, 39, 53, 59867, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>)

## Astral wrapper draft

In [18]:
class Celestial(BaseModel):
         
    timezone : str
    date : dt.date
    longitude : float
    latitude: float
    
    @property
    def info(self):
        # name and region are of no consequence to any of the calculations
        return LocationInfo(timezone=self.timezone, latitude=self.latitude, longitude=self.longitude,
                            name='undefined_name', region='undefinded_region')
    
    @property
    def location(self):
        return Location(self.info)
    
    @property
    def event(self):
        sunrise    = self.location.sunrise(self.date)
        sunset     = self.location.sunset(self.date)
        goldenhour = self.location.golden_hour(self.date)
        
        return {
            'sunrise': {'start': sunrise, 'end': sunrise},
            'sunset': {'start': sunset, 'end': sunset},
            'goldenhour': {'start': goldenhour[0], 'end': goldenhour[1]},
        }

In [19]:
test = Celestial(timezone='Europe/Berlin', date=dt.date(2021, 3, 1), longitude=13.390178, latitude=52.547302)

In [20]:
test.event['sunrise']

{'start': datetime.datetime(2021, 3, 1, 6, 52, 59, 599100, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
 'end': datetime.datetime(2021, 3, 1, 6, 52, 59, 599100, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>)}

In [21]:
# after updating the date, we get newly calculated events!
test.date = dt.date(2021, 3, 2)

In [22]:
test.event['sunrise']

{'start': datetime.datetime(2021, 3, 2, 6, 50, 45, 291119, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
 'end': datetime.datetime(2021, 3, 2, 6, 50, 45, 291119, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>)}